# FSDS Group Assessment (Group Safari)

## 1. Data Collection and Cleaning
We will use 2 different datasets:
1. Airbnb data of London (10 Dec, 2022) downloading from [InsideAirbnb](http://insideairbnb.com/get-the-data)  
2. 2011 and 2021 Census data including:
* .csv
* .csv
* .xls
* .xlsx
* ...


### 1.1 Input data and create dataframe and geodataframe

Note that all data in the Data subdirectory is ignored in the `.gitignore` file. <span style="color:red">(***We may need to change the setting of our repo later.***)</span>

The file names that are created through this script is as follows.

|Data|File name|df/gdf name|
|:---|:---|:---|
|Points|`***`|`***`|
|Trips|`***`|`***`|


#### 1.1.1 Prepare

In [1]:
# Import packages

import os
from urllib.request import urlopen
from requests import get
from urllib.parse import urlparse
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import re

<span style="color:red">For now, I am using local files, so the next coding cell won't be helpful. But I'll adjust it later to download directly using url.</span>

In [ ]:
# Download data from remote location
def cache_data(src:str, dest:str) -> str:
    """Downloads and caches a remote file locally.
    
    The function sits between the 'read' step of a pandas or geopandas
    data frame and downloading the file from a remote location. The idea
    is that it will save it locally so that you don't need to remember to
    do so yourself. Subsequent re-reads of the file will return instantly
    rather than downloading the entire file for a second or n-th itme.
    
    Parameters
    ----------
    src : str
        The remote *source* for the file, any valid URL should work.
    dest : str
        The *destination* location to save the downloaded file.
        
    Returns
    -------
    str
        A string representing the local location of the file.
    """
    url = urlparse(src)
    fn  = os.path.split(url.path)[-1]
    dfn = os.path.join(dest,fn)
    
    if not os.path.isfile(dfn):
        print(f"{dfn} not found, downloading!")
        path = os.path.split(dest)
        
        if len(path) >= 1 and path[0] != '':
            os.makedirs(os.path.join(*path), exist_ok=True)
            
        with open(dfn, "wb") as file:
            response = get(src)
            file.write(response.content)  
        print("\tDone downloading...")
    else:
        print(f"Found {dfn} locally!")
        
    return dfn

Please save data files under directory: ***fsds/group/Data***

In [13]:
os.chdir('/home/jovyan/work/Documents/casa/fsds/group')
padir = 'Data/'

#### 1.1.2 Read files used for gentrification score

In [15]:
## Population Churn
popch2011 = pd.read_csv(padir+'popchurn 11.csv', skiprows=7, header=0, skip_blank_lines=True, usecols=[
    'mnemonic',
    'Whole household lived at same address one year ago', 
    'Wholly moving household: Total']).dropna(how='all')

popch2021_in_raw = pd.read_csv(padir+'MIG009EW_LTLA_IN.csv',usecols=[
    'Lower tier local authorities code',
    'Household migration LTLA (inflow) (7 categories) code',
    'Count'])
popch2021_out_raw = pd.read_csv(padir+'MIG009EW_LTLA_OUT.csv',usecols=[
    'Migrant LTLA one year ago code', 
    'Household migration LTLA (outflow) (3 categories) code',
    'Count'])
popch2021_in = popch2021_in_raw.loc[popch2021_in_raw['Lower tier local authorities code'].astype(str).str.contains(r'^E090000[0-2][0-9]$|^E090003[0-3]$', regex=True)]
popch2021_out = popch2021_out_raw.loc[popch2021_out_raw['Migrant LTLA one year ago code'].astype(str).str.contains(r'^E090000[0-2][0-9]$|^E090003[0-3]$', regex=True)]

## Ethnic Group
eg2011 = pd.read_csv(padir+'ethnic group 2011.csv', skiprows=7, header=0, skip_blank_lines=True, usecols=[
    'mnemonic','All categories: Ethnic group','White'])
eg2021 = pd.read_csv(padir+'ethnic group 2021.csv', skiprows=6, header=0, skip_blank_lines=True, usecols=[
    'mnemonic','Total: All usual residents','White'])


     Local authority code  Year ending Dec 2001  Year ending Dec 2021
4000            E09000002               95000.0              344000.0
4001            E09000002              115000.0              390000.0
4002            E09000002              109500.0              417250.0
4003            E09000002               89995.0              345000.0
4004            E09000002               87747.5              330000.0


### housing price

In [2]:
# set the dir Pls change it
padir = 'Data/'

In [3]:
## Housing Median Price
price_med_raw = pd.read_excel(padir+'house price_median.xls',sheet_name='1a',engine='xlrd',skiprows=5,header=0,usecols=[
    'Local authority code','Year ending Dec 2001','Year ending Dec 2021'])
price_med = price_med_raw.loc[price_med_raw['Local authority code'].astype(str).str.contains(r'^E09', regex=True)]
price_med.set_index('Local authority code', inplace=True)

In [4]:
# check the values of code
unique_values = price_med.index.unique()
print(unique_values)

Index(['E09000002', 'E09000003', 'E09000004', 'E09000005', 'E09000006',
       'E09000007', 'E09000001', 'E09000008', 'E09000009', 'E09000010',
       'E09000011', 'E09000012', 'E09000013', 'E09000014', 'E09000015',
       'E09000016', 'E09000017', 'E09000018', 'E09000019', 'E09000020',
       'E09000021', 'E09000022', 'E09000023', 'E09000024', 'E09000025',
       'E09000026', 'E09000027', 'E09000028', 'E09000029', 'E09000030',
       'E09000031', 'E09000032', 'E09000033'],
      dtype='object', name='Local authority code')


In [5]:
Housing_med_df = pd.DataFrame()
Housing_med_df ['median_2011'] =price_med.loc[:, ['Year ending Dec 2001']]
Housing_med_df['median_2021'] =price_med.loc[:, ['Year ending Dec 2021']]
Housing_med_df = Housing_med_df.groupby('Local authority code')[['median_2011', 'median_2021']].median()

print(Housing_med_df)

                      median_2011  median_2021
Local authority code                          
E09000001               237500.00     797500.0
E09000002                87871.25     333750.0
E09000003               185000.00     600000.0
E09000004               119983.75     394000.0
E09000005               158656.25     509250.0
E09000006               158500.00     490000.0
E09000007               249725.00     808125.0
E09000008               125000.00     398750.0
E09000009               165000.00     520000.0
E09000010               131000.00     440250.0
E09000011               122500.00     425000.0
E09000012               161125.00     557750.0
E09000013               228000.00     707500.0
E09000014               140000.00     543500.0
E09000015               165975.00     512500.0
E09000016               123247.50     406250.0
E09000017               139750.00     428750.0
E09000018               142497.50     442125.0
E09000019               197500.00     630000.0
E09000020    

In [12]:
# load the file of average housing price 
housing_price = "house price_aver.xlsx"

# saved file name
housing_df = pd.read_excel(os.path.join(padir, housing_price),sheet_name=2,skiprows=1, header=0,index_col=0)

# set the index to datetime data
housing_df.index = pd.to_datetime(housing_df.index, format='%Y%m%d')

# set the column and index name
housing_df.columns.name = 'London_borough'
housing_df.index.name = 'year'

# check the index(year) type
print(housing_df.index.dtype)

# select the london borough data
London_housing_df = housing_df.filter(regex='^E09', axis=1)

# change the column and index location 
London_housing_df = London_housing_df.transpose()

# check the data
London_housing_df.head(3) 

datetime64[ns]


year,1995-01-01,1995-02-01,1995-03-01,1995-04-01,1995-05-01,1995-06-01,1995-07-01,1995-08-01,1995-09-01,1995-10-01,...,2022-12-01,2023-01-01,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-08-01,2023-09-01
London_borough,,,,,,,,,,,,,,,,,,,,,
E09000001,91448.98487,82202.77314,79120.70256,77101.20804,84409.14932,94900.51244,110128.04230,112329.43760,104473.10960,108038.11810,...,975240.0,963094.0,869039.0,930986.0,903718.0,958418.0,951649.0,931176.0,861107.0,807475.0
E09000002,50460.22660,51085.77983,51268.96956,53133.50526,53042.24852,53700.34831,52113.12157,52232.19868,51471.61353,51513.75760,...,355596.0,353429.0,346193.0,345288.0,348254.0,349590.0,346099.0,343430.0,344171.0,345614.0
E09000003,93284.51832,93190.16963,92247.52435,90762.87492,90258.00033,90107.23471,91441.24768,92361.31512,93273.12245,92567.38498,...,592328.0,596085.0,602336.0,596064.0,587132.0,575141.0,575829.0,583479.0,597591.0,584740.0


In [13]:
# select the data  of 2011 and 2021
housing_ave_df = pd.DataFrame()
housing_ave_df ['average_2011'] =London_housing_df.loc[:, ['2011-12-01']]
housing_ave_df ['average_2021'] =London_housing_df.loc[:, ['2012-12-01']]
housing_ave_df.head(10) 

,average_2011,average_2021
London_borough,,
E09000001,496290.8438,491528.7635
E09000002,162631.3061,167813.0248
E09000003,346652.9974,361370.6666
E09000004,199523.2257,206956.1398
E09000005,301759.5506,322170.5341
E09000006,278849.1468,288681.6021
E09000007,547656.5284,620350.6021
E09000008,217808.9162,226742.0544
E09000009,303094.5273,327287.6027


In [15]:
# link the median data and average data
total_housing_df = pd.merge(housing_ave_df,Housing_med_df, left_index=True, right_index=True)
# calculate the change of housing price
total_housing_df['Compare_2011'] = total_housing_df['median_2011']/total_housing_df['average_2011']
total_housing_df['Compare_2021'] = total_housing_df['median_2021']/total_housing_df['average_2021']
total_housing_df['Housing_Change'] = (total_housing_df['Compare_2021']-total_housing_df['Compare_2011']) / total_housing_df['Compare_2011']

In [16]:
# check the data
print(total_housing_df)

                average_2011  average_2021  median_2011  median_2021  \
London_borough                                                         
E09000001        496290.8438  4.915288e+05    237500.00     797500.0   
E09000002        162631.3061  1.678130e+05     87871.25     333750.0   
E09000003        346652.9974  3.613707e+05    185000.00     600000.0   
E09000004        199523.2257  2.069561e+05    119983.75     394000.0   
E09000005        301759.5506  3.221705e+05    158656.25     509250.0   
E09000006        278849.1468  2.886816e+05    158500.00     490000.0   
E09000007        547656.5284  6.203506e+05    249725.00     808125.0   
E09000008        217808.9162  2.267421e+05    125000.00     398750.0   
E09000009        303094.5273  3.272876e+05    165000.00     520000.0   
E09000010        245870.4032  2.549517e+05    131000.00     440250.0   
E09000011        223507.8282  2.417701e+05    122500.00     425000.0   
E09000012        318590.2533  3.426198e+05    161125.00     5577

In [ ]:
os.getcwd()
# Display the filtered DataFrame
popch2011.sample(3,random_state=7)
popch2021_in.sample(3,random_state=7)
popch2021_out.sample(3,random_state=7)
eg2011.sample(3,random_state=7)
eg2021.sample(3,random_state=7)

In [ ]:
# data source
# https://cycling.data.tfl.gov.uk/

# files saved under Data/ActiveTravelCounts
dir = 'Data/ActiveTravelCounts'
# raw files
loc_raw = '0-Count locations.csv'
central_raw = '2022-Central.csv'
inner_raw1 = '2022-Inner-Part1.csv'
inner_raw2 = '2022-Inner-Part2.csv'
outer_raw = '2022-Outer.csv'
# saved file name
location_fn = 'count_locations.geoparquet'
travelcounts_fn = 'travel_counts.parquet'

# geodataframe for points data will be saved as loc_gdf
# dataframe for counts will be saved as counts_df

# load the points data

# check if gpkg file already exists
# if not, convert the raw file into geoparquet after reading it in
if not os.path.exists(os.path.join(dir, location_fn)):
    print("Loading locations from csv and saving as geoparquet")
    loc_df = pd.read_csv(os.path.join(dir, loc_raw))
    loc_gdf = gpd.GeoDataFrame(loc_df, geometry = gpd.points_from_xy(loc_df['Easting (UK Grid)'], loc_df['Northing (UK Grid)'], crs = 'EPSG:27700'))
    # convert Functional area for monitoring into category
    loc_gdf['Functional area for monitoring'] = loc_gdf['Functional area for monitoring'].astype('category')
    loc_gdf.to_parquet(os.path.join(dir, location_fn))

# if file already there, load from gpkg
else:
    print("Loading locations from processed geoparquet")
    loc_gdf = gpd.read_parquet(os.path.join(dir, location_fn))

print("Location load complete. Use loc_gdf")

# load the travel counts data
# check if file already exists
# if not, load from csv and save the chunk before analysis

if not os.path.exists(os.path.join(dir, travelcounts_fn)):
    print("Loading counts from CSV and cleaning data")

    # load files
    cen_df = pd.read_csv(os.path.join(dir, central_raw))
    in1_df = pd.read_csv(os.path.join(dir, inner_raw1))
    in2_df = pd.read_csv(os.path.join(dir, inner_raw2))
    out_df = pd.read_csv(os.path.join(dir, outer_raw))

    # add zone
    cen_df.insert(2, 'Zone', 'Central')
    in1_df.insert(2, 'Zone', 'Inner')
    in2_df.insert(2, 'Zone', 'Inner')
    out_df.insert(2, 'Zone', 'Outer')

    # join data frames
    counts_df = pd.concat([cen_df, in1_df, in2_df, out_df])

    # clean data
    # insert datetime column in datetime format
    counts_df.insert(3, 'datetime', pd.to_datetime(counts_df['Date'] + ' ' + counts_df['Time'], dayfirst = True))
    
    # turn into categorical data
    categorical = ['Zone', 'Weather', 'Day', 'Round', 'Dir', 'Path', 'Mode']
    
    for c in categorical:
        counts_df[c] = counts_df[c].astype('category')

    # save parquet file
    counts_df.to_parquet(os.path.join(dir, travelcounts_fn))

# if file already there, load from parquet
else:
    print("Loading counts from processed parquet")
    counts_df = pd.read_parquet(os.path.join(dir, travelcounts_fn))

print("Counts load complete. Use counts_df")